# Introduction

In this notebook we demonstrate the use of **LSI (Latent Semantic Indexing)** technique of Information Retrieval context to make trace link recovery between Test Cases and Bug Reports.

We model our study as follows:

* Each bug report title, summary and description compose a single query.
* We use each use case content as an entire document that must be returned to the query made

## Import Libraries

In [1]:
import pandas as pd
import numpy as np

from sklearn.pipeline import Pipeline
from sklearn.metrics import precision_recall_fscore_support, pairwise_distances, pairwise
from sklearn.externals.joblib import Parallel, delayed
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.preprocessing import Normalizer, normalize

from enum import Enum
import pickle
import nltk

from utils import plots
from utils import firefox_dataset as fd
from utils import tokenizers as tok
from utils import aux_functions
from utils import model_evaluator as m_eval
from utils import generic_model as g_model

import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

import warnings; warnings.simplefilter('ignore')

## Load Dataset and Preprocessing

In [2]:
test_cases_df = fd.read_testcases_df()
bug_reports_df = fd.read_bugreports_df()

corpus = test_cases_df.tc_desc
query = bug_reports_df.br_desc

test_cases_names = test_cases_df.tc_name
bug_reports_names = bug_reports_df.br_name

orc = fd.read_trace_df()
orc.set_index('tc_name', inplace=True, drop=True)

In [3]:
print(bug_reports_df.shape)
bug_reports_df.head()

(35314, 12)


,Bug_Number,Summary,Platform,Component,Version,Creation_Time,Whiteboard,QA_Whiteboard,First_Comment_Text,First_Comment_Creation_Time,br_name,br_desc
0,506297,Livemarks with null site/feed uris cause sync ...,All,Sync,unspecified,2009-07-24T17:08:43Z,NaN,NaN,2009-07-24 09:54:28 FaultTolerance D...,2009-07-24T17:08:43Z,BR_506297_SRC,506297 Livemarks with null site/feed uris caus...
1,506338,Enhance Crash Recovery to better help the user,All,Session Restore,Trunk,2009-07-24T19:17:21Z,[crashkill][crashkill-metrics],NaN,When our users crash they are pretty much in t...,2009-07-24T19:17:21Z,BR_506338_SRC,506338 Enhance Crash Recovery to better help t...
2,506507,Dragging multiple bookmarks in the bookmarks s...,x86,Bookmarks & History,Trunk,2009-07-26T06:16:02Z,NaN,NaN,User-Agent: Mozilla/5.0 (Windows; U; Win...,2009-07-26T06:16:02Z,BR_506507_SRC,506507 Dragging multiple bookmarks in the book...
3,506550,Unreliable Back Button navigating nytimes.com,x86,Extension Compatibility,3.5 Branch,2009-07-26T16:12:10Z,[caused by adblock plus][platform-rel-NYTimes],NaN,User-Agent: Mozilla/5.0 (Windows; U; Win...,2009-07-26T16:12:10Z,BR_506550_SRC,506550 Unreliable Back Button navigating nytim...
4,506575,ALT + F4 when dropdown of autocomplete is open...,x86,Address Bar,3.5 Branch,2009-07-26T20:14:54Z,NaN,NaN,Pressing ALT + F4 when the autocomplete dropdo...,2009-07-26T20:14:54Z,BR_506575_SRC,506575 ALT + F4 when dropdown of autocomplete ...


In [4]:
print(test_cases_df.shape)
test_cases_df.head()

(207, 10)


,TC_Number,TestDay,Gen_Title,Crt_Nr,Title,Preconditions,Steps,Expected_Result,tc_name,tc_desc
0,1,20181221,<notificationbox> \nand\n <notification>\n cha...,1,Notification - Popup Block,NaN,1. Launch Firefox\n2. Navigate to http://www.p...,1. Firefox is successfully launched\n9. The al...,TC_1_TRG,1 20181221 <notificationbox> \nand\n <notifica...
1,2,20181221,<notificationbox> \nand\n <notification>\n cha...,2,Notification - Process Hang,NaN,"1. Launch Firefox\n2. In the URL bar, navigate...",1. Firefox is successfully launched\n2. Firefo...,TC_2_TRG,2 20181221 <notificationbox> \nand\n <notifica...
2,3,20181221,<notificationbox> \nand\n <notification>\n cha...,3,Verify Notifications appear in RTL Mode,NaN,"1. Launch Firefox\n2. In about:config, change ...",1. Firefox is successfully launched\n2.The for...,TC_3_TRG,3 20181221 <notificationbox> \nand\n <notifica...
3,4,20181221,<notificationbox> \nand\n <notification>\n cha...,4,Verify Notifications appear in High Contrast M...,NaN,"1. While the browser is in High Contrast Mode,...",1. Firefox has been launched.\n2. Firefox begi...,TC_4_TRG,4 20181221 <notificationbox> \nand\n <notifica...
4,5,20181221,<notificationbox> \nand\n <notification>\n cha...,5,Verify notifications react to differing Zoom l...,NaN,"1. While the browser is in High Contrast Mode,...",1. Firefox has been launched.\n2. Firefox begi...,TC_5_TRG,5 20181221 <notificationbox> \nand\n <notifica...


In [5]:
print(orc.shape)
orc.head()

(207, 35314)


,BR_506297_SRC,BR_506338_SRC,BR_506507_SRC,BR_506550_SRC,BR_506575_SRC,BR_506729_SRC,BR_506768_SRC,BR_506795_SRC,BR_506820_SRC,BR_506831_SRC,...,BR_1516270_SRC,BR_1516329_SRC,BR_1516358_SRC,BR_1516416_SRC,BR_1516505_SRC,BR_1516547_SRC,BR_1516582_SRC,BR_1516749_SRC,BR_1516792_SRC,BR_1516895_SRC
tc_name,,,,,,,,,,,,,,,,,,,,,
TC_1_TRG,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
TC_2_TRG,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
TC_3_TRG,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
TC_4_TRG,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
TC_5_TRG,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# LSI Model

#### Model Hyperparameters

In [6]:
class LSI_Model_Hyperp(Enum):
    NAME = 'lsi__name'
    TOP = 'lsi__top'
    SIM_MEASURE_MIN_THRESHOLD = 'lsi__sim_measure_min_threshold'
    VECTORIZER = 'lsi__vectorizer'
    VECTORIZER_STOP_WORDS = 'lsi__vectorizer__stop_words'
    VECTORIZER_TOKENIZER = 'lsi__vectorizer__tokenizer'
    VECTORIZER_USE_IDF = 'lsi__vectorizer__use_idf'
    VECTORIZER_SMOOTH_IDF = 'lsi__vectorizer__smooth_idf'
    VECTORIZER_NGRAM_RANGE = 'lsi__vectorizer__ngram_range'
    SVD_MODEL = 'lsi__svd_model'
    SVD_MODEL_N_COMPONENTS = 'lsi__svd_model__n_components'

#### Distance Functions / Similarity Measures Available

In [7]:
class SimilarityMeasure(Enum):
    COSINE = 'cosine'
    JACCARD_INDEX = 'jaccard'
    EDIT_DISTANCE = 'edit'

#### Model Definition

In [8]:
"""
params_dict = {
    'lsi__sim_measure_min_threshold' : ('cosine',.9),
    'lsi__name' : 'LSI',
    'lsi__top' : 3,
    'lsi__vectorizer' : TfidfVectorizer(),
    'lsi__vectorizer__stop_words' : 'english',
    'lsi__vectorizer__tokenizer' : Tokenizer(),
    'lsi__vectorizer__use_idf' : True,          # optional if type(Vectorizer) == TfidfVectorizer
    'lsi__vectorizer__smooth_idf' : True,       # optional if type(Vectorizer) == TfidfVectorizer
    'lsi__vectorizer__ngram_range' : (1,2),
    'lsi__svd_model' : TruncatedSVD(),
    'lsi__svd_model__n_components' : 5
}
"""
class LSI(g_model.GenericModel):
    def __init__(self, **kwargs):
        self._svd_matrix = None
        self._query_vector = None
        
        self.vectorizer = None
        self.svd_model = None
        
        super().__init__()
        
        self.set_basic_params(**kwargs)
        self.set_vectorizer(**kwargs)
        self.set_svd_model(**kwargs)
    
    def set_name(self, name):
        super().set_name(name)
    
    def set_model_gen_name(self, gen_name):
        super().set_model_gen_name(gen_name)
    
    def set_top(self, top):
        super().set_top(top)
    
    def set_sim_measure_min_threshold(self, threshold):
        super().set_sim_measure_min_threshold(threshold)
    
    def set_basic_params(self, **kwargs):
        self.set_name('LSI' if LSI_Model_Hyperp.NAME.value not in kwargs.keys() else kwargs[LSI_Model_Hyperp.NAME.value])
        self.set_sim_measure_min_threshold((SimilarityMeasure.COSINE.value,.80) if LSI_Model_Hyperp.SIM_MEASURE_MIN_THRESHOLD.value not in kwargs.keys() else kwargs[LSI_Model_Hyperp.SIM_MEASURE_MIN_THRESHOLD.value])
        self.set_top(3 if LSI_Model_Hyperp.TOP.value not in kwargs.keys() else kwargs[LSI_Model_Hyperp.TOP.value])
        self.set_model_gen_name('lsi')
    
    def set_vectorizer(self, **kwargs):
        self.vectorizer = TfidfVectorizer(stop_words='english',
                                             use_idf=True, 
                                             smooth_idf=True) if LSI_Model_Hyperp.VECTORIZER.value not in kwargs.keys() else kwargs[LSI_Model_Hyperp.VECTORIZER.value]
        
        vec_params = {key.split('__')[2]:kwargs[key] for key,val in kwargs.items() if '__vectorizer__' in key}
        self.vectorizer.set_params(**vec_params)
    
    def set_svd_model(self, **kwargs):
        self.svd_model = TruncatedSVD(n_components = 100, 
                                         algorithm = 'randomized',
                                         n_iter = 10, 
                                         random_state = 42) if LSI_Model_Hyperp.SVD_MODEL.value not in kwargs.keys() else kwargs[LSI_Model_Hyperp.SVD_MODEL.value]
        
        svd_model_params = {key.split('__')[2]:kwargs[key] for key,val in kwargs.items() if '__svd_model__' in key}
        self.svd_model.set_params(**svd_model_params)
        
    
    def recover_links(self, corpus, query, test_cases_names, bug_reports_names):
        metric = self.sim_measure_min_threshold[0]
        
        if metric == SimilarityMeasure.COSINE.value:
            return self._recover_links_cosine(corpus, query, test_cases_names, bug_reports_names)
        
        elif metric == SimilarityMeasure.JACCARD_INDEX.value:
            return self._recover_links_jaccard(corpus, query, test_cases_names, bug_reports_names)
        
        elif metric == SimilarityMeasure.EDIT_DISTANCE.value:
            return self._recover_links_edit(corpus, query, test_cases_names, bug_reports_names)
    
    def _recover_links_cosine(self, corpus, query, test_cases_names, bug_reports_names):
        svd_transformer = Pipeline([('vec', self.vectorizer), 
                            ('svd', self.svd_model)])

        self._svd_matrix = svd_transformer.fit_transform(corpus)
        self._query_vector = svd_transformer.transform(query)
        self._sim_matrix = pairwise.cosine_similarity(X=self._svd_matrix, Y=self._query_vector)
        self._sim_matrix = pd.DataFrame(data=self._sim_matrix, index=test_cases_names, columns=bug_reports_names)
        
        super()._fillUp_traceLinksDf(test_cases_names, bug_reports_names, self._sim_matrix) 
    
    def _recover_links_jaccard(self, corpus, query, test_cases_names, bug_reports_names):
        tokenizer = self.vectorizer.tokenizer
                
        corpus_tokens = [tokenizer.__call__(doc) for doc in corpus]        
        query_tokens = [tokenizer.__call__(doc) for doc in query]
        
        self._sim_matrix = pd.DataFrame(index = test_cases_names, 
                                       columns = bug_reports_names,
                                       data = np.zeros(shape=(len(test_cases_names), len(bug_reports_names)), dtype='int8'))
        
        for br_id, doc_query_tset in zip(bug_reports_names, query_tokens):
            for tc_id, doc_corpus_tset in zip(test_cases_names, corpus_tokens):
                self._sim_matrix.at[tc_id, br_id] = nltk.jaccard_distance(set(doc_corpus_tset), set(doc_query_tset))
        
        super()._fillUp_traceLinksDf(test_cases_names, bug_reports_names, self._sim_matrix) 
    
    def _recover_links_edit(self, corpus, query, test_cases_names, bug_reports_names):
        self._sim_matrix = pd.DataFrame(index = test_cases_names, 
                                       columns = bug_reports_names,
                                       data = np.zeros(shape=(len(test_cases_names), len(bug_reports_names)), dtype='int8'))
                
        for br_id, doc_query in zip(bug_reports_names, query):
            for tc_id, doc_corpus in zip(test_cases_names, corpus):
                self._sim_matrix.at[tc_id, br_id] = nltk.edit_distance(doc_corpus, doc_query)
        
        normalizer = Normalizer(copy=False).fit(self._sim_matrix.values)
        self._sim_matrix = pd.DataFrame(data=normalizer.transform(self._sim_matrix.values), index=test_cases_names, columns=bug_reports_names)
        
        super()._fillUp_traceLinksDf(test_cases_names, bug_reports_names, self._sim_matrix)
    
    def model_setup(self):
        return {"Setup" : 
                  [
                      {"Name" : self.get_name()},
                      {"Similarity Measure and Minimum Threshold" : self.get_sim_measure_min_threshold()},
                      {"Top Value" : self.get_top_value()},
                      {"SVD Model" : self.svd_model.get_params()},
                      {"Vectorizer" : self.vectorizer.get_params()},
                      {"Vectorizer Type" : type(self.vectorizer)}
                  ]
               }
        
    def get_query_vector(self):
        return self._query_vector
    
    def get_svd_matrix(self):
        return self._svd_matrix
    
    def get_vectorizer_type(self):
        return type(self.vectorizer)
    
    def get_tokenizer_type(self):
        return type(self.vectorizer.tokenizer)    
        
    def get_name(self):
        return super().get_name()

    def get_model_gen_name(self):
        return super().get_model_gen_name()
    
    def get_top_value(self):
        return super().get_top_value()
    
    def get_sim_measure_min_threshold(self):
        return super().get_sim_measure_min_threshold()
    
    def get_sim_matrix(self):
        return super().get_sim_matrix()
        
    def get_trace_links_df(self):
        return super().get_trace_links_df()
    
    def save_sim_matrix(self):
        super().save_sim_matrix()
    
    def get_model_dump_path(self):
        return super().get_model_dump_path()

## Evaluate Recovering Efficiency

In order to evaluate the efficiency of the algorithm tested (LSI), we use common metrics applied in the field of IR:

    * Precision
    * Recall
    * F1-score

### Analysis with Default Values of LSI Model

In [9]:
%%time

best_model = LSI()
best_model.recover_links(corpus, query, test_cases_names, bug_reports_names)

CPU times: user 1h 8min 24s, sys: 19min 5s, total: 1h 27min 30s
Wall time: 21min 56s


In [10]:
best_model.get_trace_links_df().shape

(207, 35314)

In [11]:
%%time

evaluator = m_eval.ModelEvaluator(orc, best_model)
evaluator.evaluate_model(verbose=True)
#evaluator.plot_precision_vs_recall()

{'Measures': {'Mean FScore of LSI': 3.7433692894004805e-05,
              'Mean Precision of LSI': 0.0007456910384927602,
              'Mean Recall of LSI': 1.9314480139230012e-05},
 'Setup': [{'Name': 'LSI'},
           {'Similarity Measure and Minimum Threshold': ('cosine', 0.8)},
           {'Top Value': 3},
           {'SVD Model': {'algorithm': 'randomized',
                          'n_components': 100,
                          'n_iter': 10,
                          'random_state': 42,
                          'tol': 0.0}},
           {'Vectorizer': {'analyzer': 'word',
                           'binary': False,
                           'decode_error': 'strict',
                           'dtype': <class 'numpy.float64'>,
                           'encoding': 'utf-8',
                           'input': 'content',
                           'lowercase': True,
                           'max_df': 1.0,
                           'max_features': None,
                       

### Quick Test Jaccard Values

In [34]:
all_hyperparams = {
    LSI_Model_Hyperp.TOP.value : [5],
    LSI_Model_Hyperp.VECTORIZER_NGRAM_RANGE.value: [(1,1)],
    LSI_Model_Hyperp.VECTORIZER.value : [TfidfVectorizer(stop_words='english', use_idf=True, smooth_idf=True)],
    LSI_Model_Hyperp.VECTORIZER_TOKENIZER.value : [tok.PorterStemmerBased_Tokenizer()],
    LSI_Model_Hyperp.SIM_MEASURE_MIN_THRESHOLD.value : [('jaccard',x) for x in [.2]],
}

hyperparams = aux_functions.generate_params_comb_list(**all_hyperparams)

print('Performing model hyperparameters search...')

def run_model(idx, **hyperp):    
    current_model = LSI(**hyperp)
    current_model.set_name('LSI_Model_{}'.format(idx))
    current_model.recover_links(corpus, query, use_cases_names, bug_reports_names)
    
    evaluator = m_eval.ModelEvaluator(orc.oracle, current_model)
    evaluator.evaluate_model()
    evaluator.dump_model()
    evaluator.dump_evaluator()
    
    return([evaluator.get_mean_precision(), 
            evaluator.get_mean_recall(),
            evaluator.get_mean_fscore(), 
            evaluator.get_model().get_name(),
            evaluator.get_model().get_top_value(),
            evaluator.get_model().get_vectorizer_type(), 
            evaluator.get_model().get_tokenizer_type(),
            evaluator.get_model().get_sim_measure_min_threshold()[0],
            evaluator.get_model().get_sim_measure_min_threshold()[1],
            evaluator.get_model().get_model_dump_path(),
            evaluator.get_evaluator_dump_path()
           ])

tasks = [(idx,hp) for idx,hp in enumerate(hyperparams)]
results = Parallel(n_jobs=-1, verbose=1)(delayed(run_model)(idx, **hp) for idx,hp in tasks)
results_df = pd.DataFrame(data=results, 
                          columns=['precision', 'recall', 'fscore', 'model_name', 'top_value', 'vectorizer', 'tokenizer', 'metric', 'metric_value', 'model_dump', 'evaluator_dump'])
results_df = results_df.astype(dtype={'model_dump' : str, 'evaluator_dump' : str})

Performing model hyperparameters search...


[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    0.1s finished


#### Evaluate

In [35]:
best_model = aux_functions.report_best_model(results_df)

------------ Report -------------------

Total of Analyzed Hyperparameters Combinations: 1

Best Model Hyperparameters Combination Found:

{'Measures': {'Mean FScore of LSI_Model_0': 0.0,
              'Mean Precision of LSI_Model_0': 0.0,
              'Mean Recall of LSI_Model_0': 0.0},
 'Setup': [{'Name': 'LSI_Model_0'},
           {'Similarity Measure and Minimum Threshold': ('jaccard', 0.2)},
           {'Top Value': 5},
           {'SVD Model': {'algorithm': 'randomized',
                          'n_components': 100,
                          'n_iter': 10,
                          'random_state': 42,
                          'tol': 0.0}},
           {'Vectorizer': {'analyzer': 'word',
                           'binary': False,
                           'decode_error': 'strict',
                           'dtype': <class 'numpy.int64'>,
                           'encoding': 'utf-8',
                           'input': 'content',
                           'lowercase': True,


In [36]:
best_model.get_sim_matrix()

artf_name,BR_4020_SRC,BR_3890_SRC,BR_3844_SRC,BR_4065_SRC,BR_3880_SRC,BR_3987_SRC,BR_4067_SRC,BR_3973_SRC,BR_3898_SRC,BR_3908_SRC,BR_4058_SRC,BR_4018_SRC,BR_4005_SRC,BR_3974_SRC
artf_name,,,,,,,,,,,,,,
UC_003_TRG,0,0,0,0,0,0,0,0,0,0,0,0,0,0
UC_007_TRG,0,0,0,0,0,0,0,0,0,0,0,0,0,0
UC_010_TRG,0,0,0,0,0,0,0,0,0,0,0,0,0,0
UC_002_TRG,0,0,0,0,0,0,0,0,0,0,0,0,0,0
UC_006_TRG,0,0,0,0,0,0,0,0,0,0,0,0,0,0
UC_004_TRG,0,0,0,0,0,0,0,0,0,0,0,0,0,0
UC_005_TRG,0,0,0,0,0,0,0,0,0,0,0,0,0,0
UC_008_TRG,0,0,0,0,0,0,0,0,0,0,0,0,0,0
UC_001_TRG,0,0,0,0,0,0,0,0,0,0,0,0,0,0


### Quick Test Edit Values

In [37]:
all_hyperparams = {
    LSI_Model_Hyperp.TOP.value : [5],
    LSI_Model_Hyperp.VECTORIZER_NGRAM_RANGE.value: [(1,1)],
    LSI_Model_Hyperp.VECTORIZER.value : [TfidfVectorizer(stop_words='english', use_idf=True, smooth_idf=True)],
    LSI_Model_Hyperp.VECTORIZER_TOKENIZER.value : [tok.PorterStemmerBased_Tokenizer()],
    LSI_Model_Hyperp.SIM_MEASURE_MIN_THRESHOLD.value : [('edit',x) for x in [.3]],
}

hyperparams = aux_functions.generate_params_comb_list(**all_hyperparams)

print('Performing model hyperparameters search...')

def run_model(idx, **hyperp):    
    current_model = LSI(**hyperp)
    current_model.set_name('LSI_Model_{}'.format(idx))
    current_model.recover_links(corpus, query, use_cases_names, bug_reports_names)
    
    evaluator = m_eval.ModelEvaluator(orc.oracle, current_model)
    evaluator.evaluate_model()
    evaluator.dump_model()
    evaluator.dump_evaluator()
    
    return([evaluator.get_mean_precision(), 
            evaluator.get_mean_recall(),
            evaluator.get_mean_fscore(), 
            evaluator.get_model().get_name(),
            evaluator.get_model().get_top_value(),
            evaluator.get_model().get_vectorizer_type(), 
            evaluator.get_model().get_tokenizer_type(),
            evaluator.get_model().get_sim_measure_min_threshold()[0],
            evaluator.get_model().get_sim_measure_min_threshold()[1],
            evaluator.get_model().get_model_dump_path(),
            evaluator.get_evaluator_dump_path()
           ])

#tasks = [(idx,hp) for idx,hp in enumerate(hyperparams)]
#results = Parallel(n_jobs=-1, verbose=1)(delayed(run_model)(idx, **hp) for idx,hp in tasks)
#results_df = pd.DataFrame(data=results, 
#                          columns=['precision', 'recall', 'fscore', 'model_name', 'top_value', 'vectorizer', 'tokenizer', 'metric', 'metric_value', 'model_dump', 'evaluator_dump'])
#results_df = results_df.astype(dtype={'model_dump' : str, 'evaluator_dump' : str})

Performing model hyperparameters search...


#### Evaluate

In [38]:
#best_model = aux_functions.report_best_model(results_df)

In [39]:
#best_model.get_sim_matrix()

In [40]:
#aux_functions.highlight_df(best_model.get_trace_links_df())

### Try Specific Model - Estimated Best Hyperparameters

##### Get Dataset Subsets

In [17]:
bugreports_subset_df = bug_reports_df[(bug_reports_df.Version == '48 Branch') | (bug_reports_df.Version == '60 Branch')].sample(15, random_state=42)
testcases_subset_df = test_cases_df[(test_cases_df.TestDay.str.contains('20161014')) | (test_cases_df.TestDay.str.contains('20161028'))].sample(10, random_state=1000)

selected_testcases = ['TC_{}_TRG'.format(tc_num) for tc_num in [13, 14, 15, 16, 17, 18]]  # should link with 48 Branch
aux_tc = test_cases_df[test_cases_df.tc_name.isin(selected_testcases)]

tc_subset_df = testcases_subset_df.append(aux_tc)
tc_subset_df.drop_duplicates(inplace=True)

corpus_subset = tc_subset_df.tc_desc
query_subset = bugreports_subset_df.br_desc
testcases_names_subset = tc_subset_df.tc_name
bug_reports_names_subset = bugreports_subset_df.br_name
orc_subset_df = orc.loc[testcases_names_subset, bug_reports_names_subset]

print('TestCases Subset Shape: {}'.format(tc_subset_df.shape))
print('BugReports Subset Shape: {}'.format(bugreports_subset_df.shape))
print('Oracle Subset Shape: {}'.format(orc_subset_df.shape))

TestCases Subset Shape: (14, 10)
BugReports Subset Shape: (15, 12)
Oracle Subset Shape: (14, 15)


#### Checking Content of Bugs

In [18]:
bugreports_subset_df[bugreports_subset_df.Version == '48 Branch'].head()

,Bug_Number,Summary,Platform,Component,Version,Creation_Time,Whiteboard,QA_Whiteboard,First_Comment_Text,First_Comment_Creation_Time,br_name,br_desc
11303,1268934,Reducing tab height causes a gap above tabs,Unspecified,Theme,48 Branch,2016-04-29T17:43:31Z,NaN,NaN,Created attachment 8747147 output.webm User A...,2016-04-29T17:43:31Z,BR_1268934_SRC,1268934 Reducing tab height causes a gap above...
12186,1282551,"CTRL+G finds text hidden in a drop down menu, ...",Unspecified,Search,48 Branch,2016-06-27T19:30:07Z,NaN,NaN,User Agent: Mozilla/5.0 (X11; Linux i686; rv:4...,2016-06-27T19:30:07Z,BR_1282551_SRC,1282551 CTRL+G finds text hidden in a drop dow...
12729,1291175,"revert new ""awesome"" bar in FF48",Unspecified,Untriaged,48 Branch,2016-08-02T05:52:12Z,NaN,NaN,User Agent: Mozilla/5.0 (Windows NT 6.1; Win64...,2016-08-02T05:52:12Z,BR_1291175_SRC,"1291175 revert new ""awesome"" bar in FF48 Unspe..."
13465,1299787,Impossible to set browser.urlbar.maxRichResult...,Unspecified,Untriaged,48 Branch,2016-09-01T12:49:52Z,NaN,NaN,User Agent: Mozilla/5.0 (X11; Ubuntu; Linux i6...,2016-09-01T12:49:52Z,BR_1299787_SRC,1299787 Impossible to set browser.urlbar.maxRi...


In [19]:
testcases_subset_df[testcases_subset_df.TestDay.str.contains('20160603')].head()

,TC_Number,TestDay,Gen_Title,Crt_Nr,Title,Preconditions,Steps,Expected_Result,tc_name,tc_desc
18,19,20160603 + 20160624 + 20161014,new awesomebar tests - awesome bar icons left,7,URL Bar - Globe Icon,NaN,1. Verify the Globe icon displays for Firefox ...,1. The globe is correctly displayed.,TC_19_TRG,19 20160603 + 20160624 + 20161014 new awesomeb...
15,16,20160603 + 20160624 + 20161014,new awesomebar tests - awesome bar search,4,Search State - Favorites,NaN,1. Ensure the star appears if the drop down di...,1. Search result has the star icon displayed\n...,TC_16_TRG,16 20160603 + 20160624 + 20161014 new awesomeb...
14,15,20160603 + 20160624 + 20161014,new awesomebar tests - awesome bar search,3,Search State - Favicons/Search Indicator,NaN,1. Ensure Favicons for sites are displayed - E...,1. Favicon or search indicator icon is display...,TC_15_TRG,15 20160603 + 20160624 + 20161014 new awesomeb...


In [23]:
all_hyperparams = {
    LSI_Model_Hyperp.SIM_MEASURE_MIN_THRESHOLD.value : [('cosine' ,x)  for x in [.80]],
    LSI_Model_Hyperp.TOP.value : [10],
    LSI_Model_Hyperp.SVD_MODEL_N_COMPONENTS.value: [100],
    LSI_Model_Hyperp.VECTORIZER_NGRAM_RANGE.value: [(1,1)],
    LSI_Model_Hyperp.VECTORIZER.value : [TfidfVectorizer(stop_words='english', use_idf=True, smooth_idf=True)],
    LSI_Model_Hyperp.VECTORIZER_TOKENIZER.value : [tok.WordNetBased_LemmaTokenizer()]
}

hyperparams = aux_functions.generate_params_comb_list(**all_hyperparams)          

print('Performing model hyperparameters search...')

def run_model(idx, **hyperp):    
    current_model = LSI(**hyperp)
    current_model.set_name('LSI_Model_{}'.format(idx))
    current_model.recover_links(corpus_subset, query_subset, testcases_names_subset, bug_reports_names_subset)
    
    evaluator = m_eval.ModelEvaluator(orc_subset_df, current_model)
    evaluator.evaluate_model()
    evaluator.dump_model()
    evaluator.dump_evaluator()
    
    return([evaluator.get_mean_precision(), 
            evaluator.get_mean_recall(),
            evaluator.get_mean_fscore(), 
            evaluator.get_model().get_name(),
            evaluator.get_model().get_top_value(),
            evaluator.get_model().get_vectorizer_type(), 
            evaluator.get_model().get_tokenizer_type(),
            evaluator.get_model().get_sim_measure_min_threshold()[0],
            evaluator.get_model().get_sim_measure_min_threshold()[1],
            evaluator.get_model().get_model_dump_path(),
            evaluator.get_evaluator_dump_path()
           ])

tasks = [(idx,hp) for idx,hp in enumerate(hyperparams)]
#results = Parallel(n_jobs=-1, verbose=1)(delayed(run_model)(idx, **hp) for idx,hp in tasks)
results = [run_model(t[0],**t[1]) for t in tasks]
results_df = pd.DataFrame(data=results, 
                          columns=['precision', 'recall', 'fscore', 'model_name', 'top_value', 'vectorizer', 'tokenizer', 'metric', 'metric_value', 'model_dump', 'evaluator_dump'])
results_df = results_df.astype(dtype={'model_dump' : str, 'evaluator_dump' : str})

bm = aux_functions.report_best_model(results_df)
bm.save_sim_matrix()

aux_functions.highlight_df(bm.get_trace_links_df())

Performing model hyperparameters search...
------------ Report -------------------

Total of Analyzed Hyperparameters Combinations: 1

Best Model Hyperparameters Combination Found:

{'Measures': {'Mean FScore of LSI_Model_0': 0.016666666666666666,
              'Mean Precision of LSI_Model_0': 0.06666666666666667,
              'Mean Recall of LSI_Model_0': 0.009523809523809523},
 'Setup': [{'Name': 'LSI_Model_0'},
           {'Similarity Measure and Minimum Threshold': ('cosine', 0.8)},
           {'Top Value': 10},
           {'SVD Model': {'algorithm': 'randomized',
                          'n_components': 100,
                          'n_iter': 10,
                          'random_state': 42,
                          'tol': 0.0}},
           {'Vectorizer': {'analyzer': 'word',
                           'binary': False,
                           'decode_error': 'strict',
                           'dtype': <class 'numpy.float64'>,
                           'encoding': 'utf-8'

br_name,BR_1268934_SRC,BR_1463735_SRC,BR_1432520_SRC,BR_1497738_SRC,BR_1463274_SRC,BR_1461828_SRC,BR_1443754_SRC,BR_1443632_SRC,BR_1282551_SRC,BR_1418983_SRC,BR_1291175_SRC,BR_1513270_SRC,BR_1450216_SRC,BR_1436749_SRC,BR_1299787_SRC
tc_name,,,,,,,,,,,,,,,
TC_166_TRG,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
TC_19_TRG,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
TC_153_TRG,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
TC_16_TRG,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
TC_161_TRG,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
TC_170_TRG,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
TC_146_TRG,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
TC_15_TRG,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
TC_150_TRG,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [24]:
aux_functions.highlight_df(orc_subset_df)

,BR_1268934_SRC,BR_1463735_SRC,BR_1432520_SRC,BR_1497738_SRC,BR_1463274_SRC,BR_1461828_SRC,BR_1443754_SRC,BR_1443632_SRC,BR_1282551_SRC,BR_1418983_SRC,BR_1291175_SRC,BR_1513270_SRC,BR_1450216_SRC,BR_1436749_SRC,BR_1299787_SRC
tc_name,,,,,,,,,,,,,,,
TC_166_TRG,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
TC_19_TRG,1,0,0,0,0,0,0,0,1,0,1,0,0,0,1
TC_153_TRG,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
TC_16_TRG,1,0,0,0,0,0,0,0,1,0,1,0,0,0,1
TC_161_TRG,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
TC_170_TRG,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
TC_146_TRG,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
TC_15_TRG,1,0,0,0,0,0,0,0,1,0,1,0,0,0,1
TC_150_TRG,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


### Find The Best Model

In [22]:
all_hyperparams = {
    LSI_Model_Hyperp.SIM_MEASURE_MIN_THRESHOLD.value : [('cosine' ,x)  for x in [.75,.85,.95]],
                                                       #[('jaccard',x)  for x in [.30,.40]] +
                                                       #[('edit',   x)  for x in [.30,.50]],
    LSI_Model_Hyperp.TOP.value : [10],
    LSI_Model_Hyperp.SVD_MODEL_N_COMPONENTS.value: [5,10],
    LSI_Model_Hyperp.VECTORIZER_NGRAM_RANGE.value: [(1,1), (1,2)],
    LSI_Model_Hyperp.VECTORIZER.value : [TfidfVectorizer(stop_words='english', use_idf=True, smooth_idf=True), 
                         CountVectorizer(stop_words='english')],
    LSI_Model_Hyperp.VECTORIZER_TOKENIZER.value : [tok.PorterStemmerBased_Tokenizer(), tok.LancasterStemmerBased_Tokenizer(), 
                                                   tok.WordNetBased_LemmaTokenizer(), tok.SnowballStemmerBased_Tokenizer()]
}

hyperparams = aux_functions.generate_params_comb_list(**all_hyperparams)          

print('Performing model hyperparameters search...')

def run_model(idx, **hyperp):    
    current_model = LSI(**hyperp)
    current_model.set_name('LSI_Model_{}'.format(idx))
    current_model.recover_links(corpus, query, test_cases_names, bug_reports_names)
    
    evaluator = m_eval.ModelEvaluator(orc.oracle, current_model)
    evaluator.evaluate_model()
    evaluator.dump_model()
    evaluator.dump_evaluator()
    
    return([evaluator.get_mean_precision(), 
            evaluator.get_mean_recall(),
            evaluator.get_mean_fscore(), 
            evaluator.get_model().get_name(),
            evaluator.get_model().get_top_value(),
            evaluator.get_model().get_vectorizer_type(), 
            evaluator.get_model().get_tokenizer_type(),
            evaluator.get_model().get_sim_measure_min_threshold()[0],
            evaluator.get_model().get_sim_measure_min_threshold()[1],
            evaluator.get_model().get_model_dump_path(),
            evaluator.get_evaluator_dump_path()
           ])

tasks = [(idx,hp) for idx,hp in enumerate(hyperparams)]
#results = Parallel(n_jobs=-1, verbose=1)(delayed(run_model)(idx, **hp) for idx,hp in tasks)
results = [run_model(t[0],**t[1]) for t in tasks]
results_df = pd.DataFrame(data=results, 
                          columns=['precision', 'recall', 'fscore', 'model_name', 'top_value', 'vectorizer', 'tokenizer', 'metric', 'metric_value', 'model_dump', 'evaluator_dump'])
results_df = results_df.astype(dtype={'model_dump' : str, 'evaluator_dump' : str})


Performing model hyperparameters search...


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
exception calling callback for <Future at 0x7fd117ffee48 state=finished raised BrokenProcessPool>
sklearn.externals.joblib.externals.loky.process_executor._RemoteTraceback: 
'''
Traceback (most recent call last):
  File "/home/guilherme/anaconda3/envs/trace-link-recovery-study/lib/python3.6/site-packages/sklearn/externals/joblib/externals/loky/process_executor.py", line 391, in _process_worker
    call_item = call_queue.get(block=True, timeout=timeout)
  File "/home/guilherme/anaconda3/envs/trace-link-recovery-study/lib/python3.6/multiprocessing/queues.py", line 113, in get
    return _ForkingPickler.loads(res)
  File "/home/guilherme/anaconda3/envs/trace-link-recovery-study/lib/python3.6/enum.py", line 135, in __new__
    enum_members = {k: classdict[k] for k in classdict._member_names}
AttributeError: 'dict' object has no attribute '_member_names'
'''

The above exception was the direct cause of the following

BrokenProcessPool: A task has failed to un-serialize. Please ensure that the arguments of the function are all picklable.

### Report

In [51]:
bm = aux_functions.report_best_model(results_df)

------------ Report -------------------

Total of Analyzed Hyperparameters Combinations: 1

Best Model Hyperparameters Combination Found:

{'Measures': {'Mean FScore of LSI_Model_0': 0.0,
              'Mean Precision of LSI_Model_0': 0.0,
              'Mean Recall of LSI_Model_0': 0.0},
 'Setup': [{'Name': 'LSI_Model_0'},
           {'Similarity Measure and Minimum Threshold': ('cosine', 0.8)},
           {'Top Value': 10},
           {'SVD Model': {'algorithm': 'randomized',
                          'n_components': 100,
                          'n_iter': 10,
                          'random_state': 42,
                          'tol': 0.0}},
           {'Vectorizer': {'analyzer': 'word',
                           'binary': False,
                           'decode_error': 'strict',
                           'dtype': <class 'numpy.float64'>,
                           'encoding': 'utf-8',
                           'input': 'content',
                           'lowercase': True

#### Save Similarity Matrix

In [22]:
bm.save_sim_matrix()

#### Best Model for TOP 3 and 5 - Cosine

In [44]:
aux_functions.print_report_top_3_and_5_v3(results_df, SimilarityMeasure.COSINE.value)

{'Measures': {'Mean FScore of LSI_Model_8': 0.5833333333333334,
              'Mean Precision of LSI_Model_8': 0.48809523809523814,
              'Mean Recall of LSI_Model_8': 0.7976190476190476},
 'Setup': [{'Name': 'LSI_Model_8'},
           {'Similarity Measure and Minimum Threshold': ('cosine', 0.75)},
           {'Top Value': 3},
           {'SVD Model': {'algorithm': 'randomized',
                          'n_components': 5,
                          'n_iter': 10,
                          'random_state': 42,
                          'tol': 0.0}},
           {'Vectorizer': {'analyzer': 'word',
                           'binary': False,
                           'decode_error': 'strict',
                           'dtype': <class 'numpy.int64'>,
                           'encoding': 'utf-8',
                           'input': 'content',
                           'lowercase': True,
                           'max_df': 1.0,
                           'max_features': None,
    

#### Best Model for TOP 3 and 5 - Jaccard

In [45]:
#aux_functions.print_report_top_3_and_5_v3(results_df, SimilarityMeasure.JACCARD_INDEX.value)

#### Best Model for Top 3 and 5 - Edit Distance

In [46]:
#aux_functions.print_report_top_3_and_5_v3(results_df, SimilarityMeasure.EDIT_DISTANCE.value)

### Plot Highlights

In [47]:
aux_functions.highlight_df(best_model.get_trace_links_df())

artf_name,BR_4020_SRC,BR_3890_SRC,BR_3844_SRC,BR_4065_SRC,BR_3880_SRC,BR_3987_SRC,BR_4067_SRC,BR_3973_SRC,BR_3898_SRC,BR_3908_SRC,BR_4058_SRC,BR_4018_SRC,BR_4005_SRC,BR_3974_SRC
artf_name,,,,,,,,,,,,,,
UC_003_TRG,0,0,0,0,0,0,0,0,0,0,0,0,0,0
UC_007_TRG,0,0,0,0,0,0,0,0,0,0,1,0,0,0
UC_010_TRG,0,0,1,0,0,1,0,0,1,1,0,0,0,0
UC_002_TRG,1,1,0,0,1,1,1,1,0,0,0,1,0,1
UC_006_TRG,1,1,0,0,1,1,1,1,0,0,0,1,0,1
UC_004_TRG,1,1,0,1,1,1,1,1,0,0,0,1,0,1
UC_005_TRG,1,1,0,0,1,1,1,1,0,0,0,1,0,1
UC_008_TRG,0,0,0,1,1,0,0,0,0,0,0,0,1,0
UC_001_TRG,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [48]:
aux_functions.highlight_df(orc.oracle)

,BR_4020_SRC,BR_3890_SRC,BR_3844_SRC,BR_4065_SRC,BR_3880_SRC,BR_3987_SRC,BR_4067_SRC,BR_3973_SRC,BR_3898_SRC,BR_3908_SRC,BR_4058_SRC,BR_4018_SRC,BR_4005_SRC,BR_3974_SRC
artf_name,,,,,,,,,,,,,,
UC_003_TRG,0,0,0,0,0,0,0,0,0,0,0,0,0,0
UC_007_TRG,0,0,0,0,0,0,0,0,0,0,1,0,0,0
UC_010_TRG,0,0,1,0,0,0,0,0,0,0,0,0,0,0
UC_002_TRG,0,0,0,0,1,0,0,1,0,0,0,0,0,0
UC_006_TRG,1,1,0,1,1,0,0,0,0,0,0,1,0,1
UC_004_TRG,0,0,0,0,0,1,0,0,0,0,0,0,0,0
UC_005_TRG,1,1,0,0,1,0,1,0,0,0,0,1,0,0
UC_008_TRG,0,0,0,0,0,0,0,0,0,0,0,0,1,0
UC_001_TRG,0,0,0,0,0,0,0,0,0,0,0,0,0,0
